In [1]:
import cifar10
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
MEMORY_LIMIT = 1024
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

In [3]:
cifar10.data_path = "data/CIFAR-10/"
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [36]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.models import load_model
import numpy as np

In [7]:
X_train = images_train.astype('float32') 
X_test = images_test.astype('float32')
# X_train = X_train / 255.0
# X_test = X_test / 255.0

### General mean-std feature scaling

In [8]:
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

In [9]:
# one hot encode outputs
y_train = np_utils.to_categorical(cls_train)
y_test = np_utils.to_categorical(cls_test)
num_classes = y_test.shape[1]

### learning-rate scheduling

In [10]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

### elu is same as relu just handles negative values also

In [11]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

### In-place data augmentation
Data generated at each epoch will be different (augmented)

In [12]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(X_train)

#### SGD not working
#### Adam - avg performance
#### Rmsprop - best

In [13]:
batch_size=64
opt = RMSprop(learning_rate=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=(X_train.shape[0] // batch_size),epochs=125, verbose=1,validation_data=(X_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
781/781 [==============================] - 86s 110ms/step - loss: 1.8953 - accuracy: 0.4217 - val_loss: 1.5083 - val_accuracy: 0.5463
Epoch 2/125
781/781 [==============================] - 67s 86ms/step - loss: 1.2679 - accuracy: 0.5878 - val_loss: 1.0574 - val_accuracy: 0.6725
Epoch 3/125
781/781 [==============================] - 67s 85ms/step - loss: 1.0732 - accuracy: 0.6535 - val_loss: 1.1451 - val_accuracy: 0.6622
Epoch 4/125
781/781 [==============================] - 67s 85ms/step - loss: 0.9771 - accuracy: 0.6882 - val_loss: 0.8671 - val_accuracy: 0.7384
Epoch 5/125
781/781 [==============================] - 67s 85ms/step - loss: 0.9050 - accuracy: 0.7159 - val_loss: 0.8341 - val_accuracy: 0.7537
Epoch 6/125
781/781 [==============================] - 67s 85ms/step - loss: 0.8616 - accuracy: 0.7339 - val_loss: 0.7769 - val_accuracy: 0.7729
Epoch 7/125
781/781 [==============================] - 67s 85ms/step - loss: 0.8253 - accuracy: 0.7472 - val_loss: 0.8040 - val_a

781/781 [==============================] - 66s 85ms/step - loss: 0.4093 - accuracy: 0.8963 - val_loss: 0.4672 - val_accuracy: 0.8805
Epoch 114/125
781/781 [==============================] - 66s 85ms/step - loss: 0.4089 - accuracy: 0.8936 - val_loss: 0.4458 - val_accuracy: 0.8893
Epoch 115/125
781/781 [==============================] - 66s 85ms/step - loss: 0.4066 - accuracy: 0.8951 - val_loss: 0.4658 - val_accuracy: 0.8845
Epoch 116/125
781/781 [==============================] - 66s 85ms/step - loss: 0.4039 - accuracy: 0.8951 - val_loss: 0.4782 - val_accuracy: 0.8784
Epoch 117/125
781/781 [==============================] - 67s 85ms/step - loss: 0.4082 - accuracy: 0.8939 - val_loss: 0.4664 - val_accuracy: 0.8842
Epoch 118/125
781/781 [==============================] - 67s 85ms/step - loss: 0.3999 - accuracy: 0.8961 - val_loss: 0.4576 - val_accuracy: 0.8835
Epoch 119/125
781/781 [==============================] - 67s 85ms/step - loss: 0.4039 - accuracy: 0.8951 - val_loss: 0.4547 - val_ac

In [24]:
model.save('model89.h5')

In [37]:
loaded_model = load_model('model89.h5')
loaded_model.evaluate(X_test, y_test)

10000/10000 [==============================] - 5s 530us/step


[0.4348409546375275, 0.8903999924659729]

In [23]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 5s 524us/step


[0.4348409546375275, 0.8903999924659729]

In [41]:
predictions = loaded_model.predict_classes(X_test)

In [49]:
cls_preds = []
for cls in predictions:
    cls_preds.append(class_names[cls])
cls_preds

['cat',
 'ship',
 'ship',
 'airplane',
 'frog',
 'frog',
 'automobile',
 'frog',
 'cat',
 'automobile',
 'airplane',
 'truck',
 'horse',
 'horse',
 'truck',
 'ship',
 'dog',
 'horse',
 'ship',
 'frog',
 'horse',
 'airplane',
 'deer',
 'truck',
 'dog',
 'bird',
 'deer',
 'airplane',
 'truck',
 'frog',
 'frog',
 'dog',
 'bird',
 'dog',
 'truck',
 'bird',
 'deer',
 'automobile',
 'truck',
 'dog',
 'deer',
 'frog',
 'dog',
 'frog',
 'airplane',
 'truck',
 'cat',
 'truck',
 'horse',
 'frog',
 'truck',
 'ship',
 'bird',
 'cat',
 'ship',
 'ship',
 'horse',
 'cat',
 'dog',
 'frog',
 'horse',
 'dog',
 'frog',
 'automobile',
 'frog',
 'bird',
 'automobile',
 'bird',
 'frog',
 'horse',
 'bird',
 'frog',
 'ship',
 'ship',
 'airplane',
 'bird',
 'truck',
 'cat',
 'cat',
 'ship',
 'ship',
 'automobile',
 'automobile',
 'horse',
 'bird',
 'dog',
 'horse',
 'horse',
 'ship',
 'truck',
 'airplane',
 'cat',
 'ship',
 'frog',
 'deer',
 'frog',
 'frog',
 'airplane',
 'airplane',
 'horse',
 'deer',
 'dog',

In [50]:
import pandas as pd
df = pd.DataFrame(cls_preds)
df.to_csv('cifar10_res.csv', index=False, header=False)